In [8]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import os
import numpy as np

#
from visualize import Visualize
from track import Track, detect_id_switch2


from tqdm import trange

#
import sleap
from tqdm import tqdm
import os
import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [11]:



#
fnames_list = '/home/cat/data/dan/id_switch/day/fnames_day.txt'

#
fnames = np.loadtxt(fnames_list, 
                    dtype='str')

#
animal_ids = np.arange(6)

#
for fname_slp in fnames:
    print ("Loading: ", fname_slp)
    
    #fname_slp = '/home/cat/data/dan/2020_07_30_00_34_28_393297/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
    track = Track(fname_slp)
    track.track_type = 'features'
    track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
    track.load_tracks()
    
    #
    fname_movie = glob.glob(fname_slp[:-10]+"*.mp4")[0]
    print ("movie: ", fname_movie)
    
    #### FIND ALL ID SWITCHES IN MOVIE ####
    all_frames = []
    
    #for animal_id in animal_ids:
    for animal_id in [2]:
        frame_ids = detect_id_switch2(track.tracks, 
                                      animal_id)
        if len(frame_ids)>0:
            for f in frame_ids:
                all_frames.append([f[0], f[1], animal_id])
    break

Loading:  /home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.slp
movie:  /home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed.mp4


finding id switches in movie: 100%|██████████████████████████| 28801/28801 [00:04<00:00, 6142.33it/s]

found:  6  switches


In [61]:
from tqdm import tqdm
    
#
print (all_frames)
search_frames = all_frames.copy()
search_frames = np.vstack((search_frames,
                           [1000000,1000000,10]))

fname = '/home/cat/data/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.slp'

#
names = ['female','male','pup1','pup2','pup3','pup4']

#
labels = sleap.load_file(fname)

labels.videos  # Use in ipython to print list of videos in the labels object
n_videos = len(labels.videos)

#
n_all=0
for video_idx in range(n_videos):
    #video_idx =   # Change this select the video of interest from the labels.video list
    video = labels.videos[video_idx]
    labeled_frames_from_video = labels.get(video)

    # Loop through all labeled frames (from a specific video)
    n_frames = 0
    ctr1=0
    for lf in labeled_frames_from_video:

        #
        if ctr1==search_frames[0][0]:
            print (ctr1, lf.has_predicted_instances)
            #
            name_switched = names[search_frames[0][2]]
            
            #
            n_inst = np.arange(lf.n_predicted_instances)
            
            #
            for k, inst in zip(n_inst, lf.predicted_instances):
                
                #
                name_predicted = lf.predicted_instances[k].track.name
                
                if name_predicted == name_switched:
                    print (name_predicted, name_switched)
                    print ("inst: ", inst)
                # delete the non-important instances:
                else:
                    lf.predicted_instances.remove(inst)
                    
            search_frames = search_frames[1:]
        else:
            for inst in lf.predicted_instances:
                
                lf.predicted_instances.remove(inst)
        ctr1+=1
        

    print (labels)
    # Save the modified slp
    new_filename = fname[:-4]+"_deleted.slp"  # Use a different path from the current slp to be safe
    labels.save_file(labeled_frames_from_video, new_filename)
print ("DONE...")

[[12959, 12960, 2], [13016, 13017, 2], [14566, 14567, 2], [14678, 14679, 2], [16416, 16417, 2], [16580, 16581, 2]]
12959 True
pup1 pup1
inst:  PredictedInstance(video=Video(filename=/scratch/cm5635/cohort2/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4, shape=(None, None, None, None), backend=MediaVideo), frame_idx=12959, points=[nose: (415.3, 400.4, 0.73), spine1: (400.4, 371.6, 0.75), spine2: (392.6, 351.8, 0.68), spine3: (396.0, 332.2, 0.78), spine4: (411.8, 320.7, 0.52), spine5: (428.4, 316.4, 0.77)], score=0.81, track=Track(spawned_on=0, name='pup1'), tracking_score=0.00)
13016 True
pup1 pup1
inst:  PredictedInstance(video=Video(filename=/scratch/cm5635/cohort2/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4, shape=(None, None, None, None), backend=MediaVideo), frame_idx=13016, points=[nose: (396.3, 411.2, 0.81), spine1: (400.9, 387.6, 0.78), spine2: (408.1, 367.6, 0.85), spine3: (416.3, 348.2, 0.71), spine4: (424.3, 328.1, 0.67), spine5: (435.

NotImplementedError: No adaptors for object type <class 'list'> (None).

female
pup1
pup4

[PredictedInstance(video=Video(filename=/scratch/cm5635/cohort2/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4, shape=(None, None, None, None), backend=MediaVideo), frame_idx=28801, points=[nose: (283.1, 547.9, 0.33), spine2: (223.5, 528.1, 0.44), spine3: (200.0, 532.1, 0.42), spine4: (183.5, 547.4, 0.51), spine5: (168.1, 552.7, 0.56)], score=0.80, track=Track(spawned_on=0, name='female'), tracking_score=0.00), PredictedInstance(video=Video(filename=/scratch/cm5635/cohort2/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4, shape=(None, None, None, None), backend=MediaVideo), frame_idx=28801, points=[spine3: (180.1, 504.1, 0.36), spine4: (208.2, 495.8, 0.46)], score=0.32, track=Track(spawned_on=0, name='pup1'), tracking_score=0.00), PredictedInstance(video=Video(filename=/scratch/cm5635/cohort2/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4, shape=(None, None, None, None), backend=MediaVideo), frame_idx=28801, point

In [8]:
####################################################
############ VISUALIZE DELETED TRACKS ##############
####################################################

fname = "/media/cat/256GB/dan/testing_track_cleanup_code/cohort2_night_1000/videos/cohort2_night.1000_deleted.slp"
labels = sleap.load_file(fname)

labels.videos  # Use in ipython to print list of videos in the labels object
n_videos = len(labels.videos)

#
locs = []
for video_idx in trange(n_videos):
    #video_idx =   # Change this select the video of interest from the labels.video list
    video = labels.videos[video_idx]
    labeled_frames_from_video = labels.get(video)

    # Loop through all labeled frames (from a specific video)
    n_frames = 0
    for ctr1, lf in enumerate(labeled_frames_from_video):

        # Loop through all user instances in the current labeled frame
        for ctr2, inst in enumerate(lf.user_instances):

            #
            remove_inst = False
            points_array = inst.points_array  # Returns a (num_nodes, 2) np.recarray
            
            #
            #print (points_array[:,0])
            #print (points_array[:,1])
            idx1 = np.where(points_array[:,0]<200)[0]
            idx2 = np.where(points_array[:,1]<200)[0]
            
            inter = list(set(idx1) & set(idx2))
            locs.append(points_array)
            
            n_all+=1
            n_frames+=1
    print (n_frames)
    
print (n_all)
locs = np.vstack(locs)
print ("locs: ", locs.shape)

#
plt.figure()
plt.scatter(locs[:,0],
            locs[:,1])
plt.xlim(0,900)
plt.ylim(0,700)
plt.show()


 33%|███▎      | 4/12 [00:00<00:00, 25.63it/s]

127
205
173
359
287


 58%|█████▊    | 7/12 [00:00<00:00, 17.16it/s]

464
512
306


 92%|█████████▏| 11/12 [00:00<00:00, 15.17it/s]

590
354
377


100%|██████████| 12/12 [00:01<00:00, 11.81it/s]

504
4258
locs:  (25548, 2)
